<a href="https://colab.research.google.com/github/canvip/canvip.github.io/blob/master/optimize_goals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import numpy as np
from scipy.optimize import linprog
# لا حاجة لاستيراد matplotlib.pyplot بعد الآن

def optimize_resources_mvp(
    objective_coefficients,
    resource_constraints_matrix,
    available_resources,
    variable_bounds=None
):
    """
    يقوم بتحسين الأهداف بموارد محدودة تحت قيود باستخدام البرمجة الخطية.

    هذا هو نموذج MVP بسيط يوضح كيفية عمل إطار التحسين.
    الهدف هو تعظيم دالة الهدف الخطية مع الالتزام بالقيود الخطية.

    المعاملات:
    objective_coefficients (list or array): معاملات دالة الهدف.
                                           ملاحظة: linprog تقوم بالتصغير، لذا إذا كنت تريد التعظيم،
                                           اجعل هذه المعاملات سالبة.
                                           (مثال: لتعظيم 30x + 40y، استخدم [-30, -40]).
    resource_constraints_matrix (list of lists or 2D array): مصفوفة معاملات قيود الموارد (<=).
                                                              كل صف يمثل قيدًا، وكل عمود يمثل متغيرًا.
    available_resources (list or array): القيم القصوى للموارد المتاحة لكل قيد.
                                        (الجانب الأيمن من قيود <=).
    variable_bounds (list of tuples, optional): حدود كل متغير.
                                                افتراضيًا، يتم تعيينها على (0, None) مما يعني أن المتغيرات غير سالبة.
                                                (مثال: [(0, None), (0, None)] لمتغيرين غير سالبين).

    العودة:
    dict: قاموس يحتوي على النتائج:
          - 'status': حالة الحل (مثال: 'optimal', 'infeasible').
          - 'optimal_values': القيم المثلى للمتغيرات.
          - 'optimal_objective_value': القيمة المثلى لدالة الهدف (بعد تحويلها للتعظيم إذا كانت المعاملات سالبة).
          - 'message': رسالة من المحلّل.
    """

    # linprog تقوم بالتصغير، لذا إذا كانت معاملات الهدف لتعظيم، يجب أن تكون سالبة
    c = np.array(objective_coefficients)

    # مصفوفة القيود (A_ub * x <= b_ub)
    A_ub = np.array(resource_constraints_matrix)
    b_ub = np.array(available_resources)

    # حدود المتغيرات (افتراضيًا غير سالبة)
    if variable_bounds is None:
        variable_bounds = [(0, None)] * len(c)

    # حل مشكلة البرمجة الخطية
    result = linprog(c, A_ub=A_ub, b_ub=b_ub, bounds=variable_bounds, method='highs')

    # تفسير النتائج
    if result.success:
        optimal_values = result.x
        # إذا كان الهدف هو التعظيم، يجب تحويل القيمة مرة أخرى
        # في حالة التصغير (معاملات موجبة)، لا نغير -result.fun
        optimal_objective_value = -result.fun if np.all(c < 0) else result.fun
        status = 'optimal'
    else:
        optimal_values = None
        optimal_objective_value = None
        status = result.status

    return {
        'status': status,
        'optimal_values': optimal_values,
        'optimal_objective_value': optimal_objective_value,
        'message': result.message
    }

def print_ascii_bar_chart(labels, values, title=""):
    """
    يطبع رسم بياني شريطي نصي (ASCII art).
    """
    print(f"\n--- {title} ---")
    if not values:
        print("لا توجد بيانات لعرضها.")
        return

    max_label_length = max(len(label) for label in labels)
    max_value = max(values)

    if max_value == 0: # تجنب القسمة على صفر
        scale = 0
    else:
        scale = 50 / max_value # لتحديد طول الشريط الأقصى بـ 50 حرفًا

    for i, label in enumerate(labels):
        value = values[i]
        bar_length = int(value * scale)
        bar = '█' * bar_length # استخدم حرف مربع لتمثيل الشريط
        print(f"{label.ljust(max_label_length)} | {bar} {value:.2f}")
    print("-" * (max_label_length + 55)) # خط فاصل

def predict_future_demand_ai(historical_data=None):
    """
    دالة محاكاة لنموذج ذكاء اصطناعي يتنبأ بالطلب المستقبلي ويقدم مقترحات.
    في سيناريو MVP، يمكن أن تكون دالة بسيطة أو قيمة ثابتة.
    في تطبيق حقيقي، ستكون هذه دالة تستخدم نموذج تعلم آلي (مثل الانحدار الخطي، الشبكات العصبية)
    تم تدريبه على بيانات تاريخية معقدة.
    """
    print("\n--- تشغيل نموذج الذكاء الاصطناعي للتنبؤ بالطلب المستقبلي وتقديم الرؤى ---")

    if historical_data is None or not historical_data:
        # إذا لم تتوفر بيانات تاريخية، نستخدم افتراضًا
        historical_data = [140, 150, 145, 155, 160] # بيانات طلب تاريخية افتراضية

    # محاكاة تحليل البيانات التاريخية
    avg_demand = np.mean(historical_data)
    demand_trend = (historical_data[-1] - historical_data[0]) / len(historical_data) if len(historical_data) > 1 else 0

    # لغرض MVP، سنفترض طلبًا مستقبليًا يعتمد على متوسط واتجاه بسيط
    predicted_demand = int(avg_demand + demand_trend * 2) # توقع بناءً على المتوسط والاتجاه

    print(f"البيانات التاريخية للطلب: {historical_data}")
    print(f"متوسط الطلب التاريخي: {avg_demand:.2f} وحدة")
    print(f"اتجاه الطلب (لكل فترة): {demand_trend:.2f} وحدة")
    print(f"الطلب المستقبلي المتوقع بواسطة الذكاء الاصطناعي: {predicted_demand} وحدة")

    # مقترحات وحلول من الذكاء الاصطناعي
    suggestions = []
    suggestions.append("1. تحليل البيانات التاريخية للطلب يظهر اتجاهًا تصاعديًا طفيفًا.")
    suggestions.append(f"2. بناءً على التنبؤ بـ {predicted_demand} وحدة، يُنصح بمراجعة سعة المسارات.")

    if predicted_demand > 100: # إذا كان الطلب المتوقع أعلى من سعة المسار 1
        suggestions.append("3. قد تحتاج إلى زيادة الاعتماد على المسار 2، أو التفكير في توسيع سعة المسار 1.")
    if predicted_demand > 200: # مثال على اقتراح أكثر جرأة لطلب أعلى
        suggestions.append("4. يُقترح استكشاف خيارات شحن جديدة أو التفاوض على عقود أفضل مع الموردين الحاليين لتلبية الطلب المتزايد.")

    print("\n--- مقترحات الذكاء الاصطناعي للمساعدة في اتخاذ القرار ---")
    for s in suggestions:
        print(s)

    return predicted_demand

def assess_and_mitigate_risks_ai(optimal_solution, predicted_demand, path_costs, path_capacities):
    """
    دالة محاكاة لنموذج ذكاء اصطناعي يقوم بتقييم المخاطر وتقديم استراتيجيات التخفيف.
    في تطبيق حقيقي، يمكن أن يشمل هذا تحليل السيناريوهات، محاكاة مونت كارلو،
    أو نماذج تعلم آلي لتحديد نقاط الضعف.
    """
    print("\n--- تشغيل نموذج الذكاء الاصطناعي لإدارة المخاطر ---")
    risks = []
    mitigations = []
    numerical_insights = {}

    # المخاطرة 1: الاعتماد المفرط على مسار واحد
    if optimal_solution is not None:
        total_shipped = sum(optimal_solution)
        if total_shipped > 0:
            path1_reliance_percent = (optimal_solution[0] / total_shipped) * 100
            path2_reliance_percent = (optimal_solution[1] / total_shipped) * 100

            numerical_insights['path1_reliance_percent'] = path1_reliance_percent
            numerical_insights['path2_reliance_percent'] = path2_reliance_percent

            if path1_reliance_percent > 80: # إذا كان أكثر من 80% من الشحنات تمر عبر المسار 1
                risks.append(f"1. اعتماد كبير على المسار 1 ({path1_reliance_percent:.2f}%), مما يزيد من المخاطر في حالة تعطله.")
                mitigations.append("- تنويع الشحنات قدر الإمكان بين المسارات لتقليل المخاطر (قد يقلل الاعتماد بنسبة 5-10%).")
                mitigations.append("- وضع خطة طوارئ للمسار 1 (بدائل، عقود احتياطية).")
            elif path2_reliance_percent > 80: # إذا كان أكثر من 80% من الشحنات تمر عبر المسار 2
                risks.append(f"1. اعتماد كبير على المسار 2 ({path2_reliance_percent:.2f}%), مما يزيد من المخاطر في حالة تعطله.")
                mitigations.append("- تنويع الشحنات قدر الإمكان بين المسارات لتقليل المخاطر (قد يقلل الاعتماد بنسبة 5-10%).")
                mitigations.append("- وضع خطة طوارئ للمسار 2 (بدائل، عقود احتياطية).")

    # المخاطرة 2: عدم كفاية السعة لتلبية الطلب المتوقع
    total_capacity = sum(path_capacities)
    if predicted_demand > total_capacity:
        capacity_deficit = predicted_demand - total_capacity
        numerical_insights['capacity_deficit'] = capacity_deficit
        risks.append(f"2. السعة الإجمالية للمسارات ({total_capacity} وحدة) أقل من الطلب المتوقع ({predicted_demand} وحدة) بعجز قدره {capacity_deficit} وحدة.")
        mitigations.append(f"- التفاوض على زيادة السعة مع الموردين الحاليين (يمكن أن يزيد السعة بنسبة 10-20%).")
        mitigations.append("- البحث عن مسارات شحن إضافية أو شركاء جدد.")
        mitigations.append("- النظر في تعديل تواريخ التسليم أو الطلبات مع العملاء.")

    # المخاطرة 3: ارتفاع تكاليف الشحن (إذا كانت التكلفة المثلى لا تزال مرتفعة)
    # هذا يتطلب معرفة مسبقة بحدود التكلفة المقبولة
    # لغرض MVP، سنفترض عتبة معينة
    cost_threshold = 650 # عتبة افتراضية
    if optimal_solution is not None:
        current_optimal_cost = optimal_solution[0] * path_costs[0] + optimal_solution[1] * path_costs[1]
        if current_optimal_cost > cost_threshold:
            cost_exceedance = current_optimal_cost - cost_threshold
            numerical_insights['cost_exceedance'] = cost_exceedance
            risks.append(f"3. التكاليف المثلى للشحن ({current_optimal_cost:.2f}$) لا تزال مرتفعة وتتجاوز العتبة المقبولة ({cost_threshold}$) بمقدار {cost_exceedance:.2f}$.")
            mitigations.append("- إعادة تقييم هياكل التكلفة للمسارات الحالية (قد تخفض التكلفة بنسبة 5%).")
            mitigations.append("- استكشاف خيارات الشحن بالجملة أو عقود طويلة الأجل (قد تخفض التكلفة بنسبة 10-15%).")

    if not risks:
        print("لا توجد مخاطر كبيرة تم تحديدها بناءً على التحليل الحالي.")
    else:
        print("\n--- المخاطر المحددة بواسطة الذكاء الاصطناعي (رقمية) ---")
        for key, value in numerical_insights.items():
            if 'percent' in key:
                print(f"- {key.replace('_', ' ')}: {value:.2f}%")
            else:
                print(f"- {key.replace('_', ' ')}: {value:.2f} وحدة/دولار")

        print("\n--- المخاطر المحددة بواسطة الذكاء الاصطناعي (وصفية) ---")
        for r in risks:
            print(r)

        print("\n--- استراتيجيات التخفيف المقترحة (مع تأثيرات تقديرية) ---")
        for m in mitigations:
            print(m)

    return numerical_insights # إرجاع الرؤى الرقمية للمخاطر

# مثال على الاستخدام: تحسين سلسلة التوريد
# شركة صناعية تحتاج إلى شحن عدد معين من المنتجات (سيتم تحديده بواسطة AI). لديها خياران للشحن:
# - المسار 1: التكلفة 5 دولارات للوحدة، يستغرق 3 أيام للوحدة. السعة القصوى 100 وحدة.
# - المسار 2: التكلفة 4 دولارات للوحدة، يستغرق 5 أيام للوحدة. السعة القصوى 120 وحدة.

# الهدف: تقليل إجمالي تكاليف الشحن مع ضمان متوسط وقت تسليم لا يتجاوز 4 أيام.

# تعريف المشكلة:
# المتغيرات: x1 = عدد الوحدات المشحونة عبر المسار 1، x2 = عدد الوحدات المشحونة عبر المسار 2

# دالة الهدف (لتقليل 5x1 + 4x2):
objective_coeffs = [5, 4] # معاملات التكلفة (للتصغير، لذا تبقى موجبة)
path_costs = [5, 4] # تكاليف المسارات لاستخدامها في تقييم المخاطر

# استخدام الذكاء الاصطناعي للتنبؤ بالطلب الإجمالي وتقديم رؤى
# يمكن تمرير بيانات تاريخية حقيقية هنا
sample_historical_demand = [130, 140, 150, 155, 165] # بيانات طلب تاريخية فعلية
total_units_to_ship_ai = predict_future_demand_ai(historical_data=sample_historical_demand)

# قيود الموارد (A_ub * x <= b_ub):
# 1. تلبية الطلب الإجمالي (x1 + x2 >= total_units_to_ship_ai) -> -x1 - x2 <= -total_units_to_ship_ai
# 2. متوسط وقت التسليم (3x1 + 5x2) / (x1 + x2) <= 4
#    يمكن تحويلها إلى: 3x1 + 5x2 <= 4x1 + 4x2
#    أو: (3-4)x1 + (5-4)x2 <= 0
#    أو: -x1 + x2 <= 0
# 3. سعة المسار 1 (x1 <= 100)
# 4. سعة المسار 2 (x2 <= 120)
resource_matrix = [
    [-1, -1],  # قيد تلبية الطلب (>=)
    [-1, 1],   # قيد متوسط وقت التسليم (<=)
    [1, 0],    # قيد سعة المسار 1 (<=)
    [0, 1]     # قيد سعة المسار 2 (<=)

]
# تحديث قيد الطلب بناءً على توقع الذكاء الاصطناعي
available_resources_limits = [-total_units_to_ship_ai, 0, 100, 120] # حدود القيود
path_capacities = [100, 120] # سعات المسارات لاستخدامها في تقييم المخاطر

# حدود المتغيرات (x1 >= 0, x2 >= 0):
bounds = [(0, None), (0, None)] # (الحد الأدنى، الحد الأقصى)

# تشغيل المحسّن
results = optimize_resources_mvp(
    objective_coeffs,
    resource_matrix,
    available_resources_limits,
    bounds
)

# عرض النتائج
print("\n--- نتائج تحسين سلسلة التوريد (MVP) ---")
print(f"الحالة: {results['status']}")

if results['status'] == 'optimal':
    print(f"القيم المثلى للمتغيرات (عدد الوحدات المشحونة):")
    print(f"  الوحدات عبر المسار 1 (x1): {results['optimal_values'][0]:.2f} وحدة")
    print(f"  الوحدات عبر المسار 2 (x2): {results['optimal_values'][1]:.2f} وحدة")
    print(f"إجمالي التكلفة الدنيا للشحن: ${results['optimal_objective_value']:.2f}")

    # حساب التكلفة قبل التحسين (سيناريو افتراضي غير محسّن)
    # نفترض أننا نستخدم المسار الأرخص (المسار 2) لأقصى سعة ثم المسار 1 للباقي،
    # دون مراعاة قيد متوسط وقت التسليم بشكل صارم.
    # إجمالي الوحدات المطلوبة: total_units_to_ship_ai
    # سعة المسار 2: 120 وحدة
    # سعة المسار 1: 100 وحدة

    # سيناريو أساسي غير محسّن: نملأ المسار 2 أولاً ثم المسار 1
    units_via_path2_unoptimized = min(120, total_units_to_ship_ai) # نستخدم المسار 2 بالكامل إذا كان الطلب يسمح
    units_via_path1_unoptimized = total_units_to_ship_ai - units_via_path2_unoptimized # الباقي عبر المسار 1

    # إذا تجاوزت الوحدات المتبقية سعة المسار 1، فهذا السيناريو غير ممكن
    if units_via_path1_unoptimized > 100:
        # هذا يعني أن السيناريو الأساسي البسيط قد لا يكون واقعيا إذا كانت السعة غير كافية
        # في هذه الحالة، سنفترض تكلفة أعلى لسيناريو غير محسّن كقيمة مرجعية
        cost_before_optimization = (100 * 5) + (120 * 4) # تكلفة ملء المسارين بالكامل كقيمة مرجعية
        print("\nملاحظة: السيناريو غير المحسّن الافتراضي تجاوز سعة المسار 1 أو 2. تم استخدام قيمة تقديرية للتكلفة قبل التحسين.")
    else:
        cost_before_optimization = (units_via_path1_unoptimized * 5) + (units_via_path2_unoptimized * 4)

    print(f"إجمالي التكلفة قبل التحسين (تقديري): ${cost_before_optimization:.2f}")

    # عرض النتائج كرسم بياني نصي
    labels = ['التكلفة قبل التحسين', 'التكلفة بعد التحسين']
    costs = [cost_before_optimization, results['optimal_objective_value']]
    print_ascii_bar_chart(labels, costs, "مقارنة تكاليف الشحن")

    # تشغيل إدارة المخاطر بعد الحصول على الحل الأمثل
    risk_insights = assess_and_mitigate_risks_ai(
        optimal_solution=results['optimal_values'],
        predicted_demand=total_units_to_ship_ai,
        path_costs=path_costs,
        path_capacities=path_capacities
    )

else:
    print(f"لم يتم العثور على حل أمثل. الرسالة: {results['message']}")

print("\n--- شرح إضافي ---")
print("يوضح هذا النموذج كيف يمكن لنظامنا أن يحدد أفضل طريقة لتوزيع الشحنات بين المسارات المتاحة لتقليل التكاليف، مع ضمان تلبية الطلب وعدم تجاوز متوسط وقت تسليم محدد.")
print("الرسم البياني النصي أعلاه يوضح بوضوح التوفير في التكاليف الذي يمكن تحقيقه من خلال تطبيق نموذج التحسين الخاص بنا مقارنة بسيناريو شحن غير محسّن.")
print("يمكن توسيع هذا المفهوم ليشمل عوامل أكثر تعقيدًا مثل أنواع الشحنات المختلفة، وتكاليف الصيانة، وجدولة المركبات، مما يوفر تحسينًا كبيرًا في عمليات سلسلة التوريد.")
print("\n--- دمج الذكاء الاصطناعي ---")
print("في هذا التحديث، قمنا بدمج دالة 'predict_future_demand_ai' التي تحاكي نموذج ذكاء اصطناعي يتنبأ بالطلب المستقبلي بناءً على البيانات التاريخية. هذا الطلب المتوقع يغذي بعد ذلك نموذج التحسين، مما يوضح كيف يمكن للذكاء الاصطناعي أن يوفر مدخلات ديناميكية للتحسين، مما يجعله أكثر استجابة للظروف المتغيرة. كما تقوم الدالة بتقديم مقترحات ورؤى بناءً على تحليلها لمساعدة في اتخاذ القرار الاستراتيجي.")
print("\n--- إدارة المخاطر المدعومة بالذكاء الاصطناعي ---")
print("تم إضافة دالة 'assess_and_mitigate_risks_ai' التي تقوم بمحاكاة تحليل المخاطر بناءً على الحل الأمثل والطلب المتوقع. تقوم هذه الدالة بتحديد المخاطر المحتملة (مثل الاعتماد المفرط على مسار واحد أو عدم كفاية السعة أو ارتفاع التكاليف) وتقديم استراتيجيات تخفيف مقترحة، مع تضمين رؤى رقمية لتقييم المخاطر بشكل كمي، مما يوفر رؤى قيمة لاتخاذ قرارات أكثر قوة واستدامة.")



--- تشغيل نموذج الذكاء الاصطناعي للتنبؤ بالطلب المستقبلي وتقديم الرؤى ---
البيانات التاريخية للطلب: [130, 140, 150, 155, 165]
متوسط الطلب التاريخي: 148.00 وحدة
اتجاه الطلب (لكل فترة): 7.00 وحدة
الطلب المستقبلي المتوقع بواسطة الذكاء الاصطناعي: 162 وحدة

--- مقترحات الذكاء الاصطناعي للمساعدة في اتخاذ القرار ---
1. تحليل البيانات التاريخية للطلب يظهر اتجاهًا تصاعديًا طفيفًا.
2. بناءً على التنبؤ بـ 162 وحدة، يُنصح بمراجعة سعة المسارات.
3. قد تحتاج إلى زيادة الاعتماد على المسار 2، أو التفكير في توسيع سعة المسار 1.

--- نتائج تحسين سلسلة التوريد (MVP) ---
الحالة: optimal
القيم المثلى للمتغيرات (عدد الوحدات المشحونة):
  الوحدات عبر المسار 1 (x1): 81.00 وحدة
  الوحدات عبر المسار 2 (x2): 81.00 وحدة
إجمالي التكلفة الدنيا للشحن: $729.00
إجمالي التكلفة قبل التحسين (تقديري): $690.00

--- مقارنة تكاليف الشحن ---
التكلفة قبل التحسين | ███████████████████████████████████████████████ 690.00
التكلفة بعد التحسين | ██████████████████████████████████████████████████ 729.00
--------------------------------

In [11]:
import numpy as np
from scipy.optimize import linprog
# لا حاجة لاستيراد matplotlib.pyplot بعد الآن

def optimize_resources_mvp(
    objective_coefficients,
    resource_constraints_matrix,
    available_resources,
    variable_bounds=None,
    constraint_types=None # 'eq' for equality, 'ub' for upper bound
):
    """
    يقوم بتحسين الأهداف بموارد محدودة تحت قيود باستخدام البرمجة الخطية.

    هذا هو نموذج MVP بسيط يوضح كيفية عمل إطار التحسين.
    الهدف هو تعظيم دالة الهدف الخطية مع الالتزام بالقيود الخطية.

    المعاملات:
    objective_coefficients (list or array): معاملات دالة الهدف.
                                           ملاحظة: linprog تقوم بالتصغير، لذا إذا كنت تريد التعظيم،
                                           اجعل هذه المعاملات سالبة.
                                           (مثال: لتعظيم 30x + 40y، استخدم [-30, -40]).
    resource_constraints_matrix (list of lists or 2D array): مصفوفة معاملات قيود الموارد.
                                                              كل صف يمثل قيدًا، وكل عمود يمثل متغيرًا.
    available_resources (list or array): القيم القصوى للموارد المتاحة لكل قيد.
                                        (الجانب الأيمن من القيود).
    variable_bounds (list of tuples, optional): حدود كل متغير.
                                                افتراضيًا، يتم تعيينها على (0, None) مما يعني أن المتغيرات غير سالبة.
                                                (مثال: [(0, None), (0, None)] لمتغيرين غير سالبين).
    constraint_types (list of str, optional): نوع كل قيد في resource_constraints_matrix.
                                              'ub' لـ <= (أقل من أو يساوي)، 'eq' لـ == (يساوي).
                                              الافتراضي هو 'ub' للجميع.

    العودة:
    dict: قاموس يحتوي على النتائج:
          - 'status': حالة الحل (مثال: 'optimal', 'infeasible').
          - 'optimal_values': القيم المثلى للمتغيرات.
          - 'optimal_objective_value': القيمة المثلى لدالة الهدف (بعد تحويلها للتعظيم إذا كانت المعاملات سالبة).
          - 'message': رسالة من المحلّل.
    """

    c = np.array(objective_coefficients)
    A_ub = []
    b_ub = []
    A_eq = []
    b_eq = []

    if constraint_types is None:
        constraint_types = ['ub'] * len(resource_constraints_matrix)

    for i, row in enumerate(resource_constraints_matrix):
        if constraint_types[i] == 'ub':
            A_ub.append(row)
            b_ub.append(available_resources[i])
        elif constraint_types[i] == 'eq':
            A_eq.append(row)
            b_eq.append(available_resources[i])
        else:
            raise ValueError("Unsupported constraint type. Use 'ub' or 'eq'.")

    A_ub = np.array(A_ub) if A_ub else None
    b_ub = np.array(b_ub) if b_ub else None
    A_eq = np.array(A_eq) if A_eq else None
    b_eq = np.array(b_eq) if b_eq else None

    if variable_bounds is None:
        variable_bounds = [(0, None)] * len(c)

    result = linprog(c, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq, bounds=variable_bounds, method='highs')

    if result.success:
        optimal_values = result.x
        optimal_objective_value = -result.fun if np.all(c < 0) else result.fun
        status = 'optimal'
    else:
        optimal_values = None
        optimal_objective_value = None
        status = result.status

    return {
        'status': status,
        'optimal_values': optimal_values,
        'optimal_objective_value': optimal_objective_value,
        'message': result.message
    }

def print_ascii_bar_chart(labels, values, title=""):
    """
    يطبع رسم بياني شريطي نصي (ASCII art).
    """
    print(f"\n--- {title} ---")
    if not values:
        print("لا توجد بيانات لعرضها.")
        return

    max_label_length = max(len(label) for label in labels)
    max_value = max(values)

    if max_value == 0:
        scale = 0
    else:
        scale = 50 / max_value

    for i, label in enumerate(labels):
        value = values[i]
        bar_length = int(value * scale)
        bar = '█' * bar_length
        print(f"{label.ljust(max_label_length)} | {bar} {value:.2f}")
    print("-" * (max_label_length + 55))

def simulate_ai_insight(scenario_type, historical_data=None):
    """
    دالة محاكاة لنموذج ذكاء اصطناعي يقدم رؤى وتنبؤات بناءً على نوع السيناريو.
    """
    print(f"\n--- تشغيل نموذج الذكاء الاصطناعي لرؤى {scenario_type} ---")
    insights = {}
    suggestions = []

    if scenario_type == "demand_prediction":
        if historical_data is None or not historical_data:
            historical_data = [140, 150, 145, 155, 160] # بيانات طلب تاريخية افتراضية

        avg_data = np.mean(historical_data)
        data_trend = (historical_data[-1] - historical_data[0]) / len(historical_data) if len(historical_data) > 1 else 0
        predicted_value = int(avg_data + data_trend * 2)

        insights['historical_data'] = historical_data
        insights['average_historical_value'] = avg_data
        insights['trend_per_period'] = data_trend
        insights['predicted_value'] = predicted_value

        print(f"البيانات التاريخية: {historical_data}")
        print(f"المتوسط التاريخي: {avg_data:.2f} وحدة")
        print(f"الاتجاه (لكل فترة): {data_trend:.2f} وحدة")
        print(f"القيمة المتوقعة بواسطة الذكاء الاصطناعي: {predicted_value} وحدة")

        suggestions.append("1. تحليل البيانات التاريخية يظهر اتجاهًا تصاعديًا طفيفًا.")
        suggestions.append(f"2. بناءً على التنبؤ بـ {predicted_value} وحدة، يُنصح بمراجعة سعة المسارات.") # تم التعديل هنا
        if predicted_value > 100:
            suggestions.append("3. قد تحتاج إلى زيادة الاعتماد على المسار 2، أو التفكير في توسيع سعة المسار 1.")
        if predicted_value > 200:
            suggestions.append("4. يُقترح استكشاف خيارات شحن جديدة أو التفاوض على عقود أفضل مع الموردين الحاليين لتلبية الطلب المتزايد.")

    elif scenario_type == "marketing_roi_potential":
        # محاكاة بيانات الأداء التاريخي للقنوات التسويقية (ROI لكل 1000 دولار استثمار)
        # ROI = (إجمالي المبيعات - تكلفة التسويق) / تكلفة التسويق
        # هنا سنستخدم "المبيعات المتوقعة لكل 1000 دولار"
        channel_performance = {
            'social_media_sales_per_k': 1500, # $1500 مبيعات لكل $1000 استثمار
            'paid_search_sales_per_k': 2000,
            'email_sales_per_k': 1000
        }

        insights['channel_performance_metrics'] = channel_performance

        # توقع إجمالي المبيعات المحتملة بناءً على الميزانية المتاحة وأداء القناة
        # هذا مجرد تقدير مبدئي قبل التحسين
        total_budget_k = 500 # 500 ألف دولار
        estimated_total_sales_potential = (
            channel_performance['social_media_sales_per_k'] * (total_budget_k / 3) +
            channel_performance['paid_search_sales_per_k'] * (total_budget_k / 3) +
            channel_performance['email_sales_per_k'] * (total_budget_k / 3)
        ) / 1000 # تحويل إلى ملايين الدولارات

        insights['estimated_total_sales_potential_M'] = estimated_total_sales_potential

        print(f"الأداء المتوقع للقنوات (مبيعات لكل 1000$ استثمار): {channel_performance}")
        print(f"إجمالي المبيعات المحتملة المقدرة (قبل التحسين الأمثل): ${estimated_total_sales_potential:.2f} مليون")

        suggestions.append("1. البحث المدفوع يظهر أعلى عائد محتمل لكل دولار مستثمر.")
        suggestions.append("2. يمكن تحسين إجمالي المبيعات بتوزيع الميزانية بناءً على أداء القناة.")
        suggestions.append("3. يُنصح بمراقبة أداء القنوات باستمرار لتعديل التخصيص.")

    print("\n--- مقترحات الذكاء الاصطناعي للمساعدة في اتخاذ القرار ---")
    for s in suggestions:
        print(s)

    return insights

def assess_and_mitigate_risks_ai(scenario_type, optimal_solution, current_params, historical_data=None):
    """
    دالة محاكاة لنموذج ذكاء اصطناعي يقوم بتقييم المخاطر وتقديم استراتيجيات التخفيف.
    """
    print(f"\n--- تشغيل نموذج الذكاء الاصطناعي لإدارة المخاطر لـ {scenario_type} ---")
    risks = []
    mitigations = []
    numerical_insights = {}

    if scenario_type == "supply_chain":
        predicted_demand = current_params['predicted_demand']
        path_costs = current_params['path_costs']
        path_capacities = current_params['path_capacities']

        # المخاطرة 1: الاعتماد المفرط على مسار واحد
        if optimal_solution is not None:
            total_shipped = sum(optimal_solution)
            if total_shipped > 0:
                path1_reliance_percent = (optimal_solution[0] / total_shipped) * 100
                path2_reliance_percent = (optimal_solution[1] / total_shipped) * 100

                numerical_insights['path1_reliance_percent'] = path1_reliance_percent
                numerical_insights['path2_reliance_percent'] = path2_reliance_percent

                if path1_reliance_percent > 80:
                    risks.append(f"1. اعتماد كبير على المسار 1 ({path1_reliance_percent:.2f}%), مما يزيد من المخاطر في حالة تعطله.")
                    mitigations.append("- تنويع الشحنات قدر الإمكان بين المسارات لتقليل المخاطر (قد يقلل الاعتماد بنسبة 5-10%).")
                    mitigations.append("- وضع خطة طوارئ للمسار 1 (بدائل، عقود احتياطية).")
                elif path2_reliance_percent > 80:
                    risks.append(f"1. اعتماد كبير على المسار 2 ({path2_reliance_percent:.2f}%), مما يزيد من المخاطر في حالة تعطله.")
                    mitigations.append("- تنويع الشحنات قدر الإمكان بين المسارات لتقليل المخاطر (قد يقلل الاعتماد بنسبة 5-10%).")
                    mitigations.append("- وضع خطة طوارئ للمسار 2 (بدائل، عقود احتياطية).")

        # المخاطرة 2: عدم كفاية السعة لتلبية الطلب المتوقع
        total_capacity = sum(path_capacities)
        if predicted_demand > total_capacity:
            capacity_deficit = predicted_demand - total_capacity
            numerical_insights['capacity_deficit'] = capacity_deficit
            risks.append(f"2. السعة الإجمالية للمسارات ({total_capacity} وحدة) أقل من الطلب المتوقع ({predicted_demand} وحدة) بعجز قدره {capacity_deficit} وحدة.")
            mitigations.append(f"- التفاوض على زيادة السعة مع الموردين الحاليين (يمكن أن يزيد السعة بنسبة 10-20%).")
            mitigations.append("- البحث عن مسارات شحن إضافية أو شركاء جدد.")
            mitigations.append("- النظر في تعديل تواريخ التسليم أو الطلبات مع العملاء.")

        # المخاطرة 3: ارتفاع تكاليف الشحن (إذا كانت التكلفة المثلى لا تزال مرتفعة)
        cost_threshold = 650
        if optimal_solution is not None:
            current_optimal_cost = optimal_solution[0] * path_costs[0] + optimal_solution[1] * path_costs[1]
            if current_optimal_cost > cost_threshold:
                cost_exceedance = current_optimal_cost - cost_threshold
                numerical_insights['cost_exceedance'] = cost_exceedance
                risks.append(f"3. التكاليف المثلى للشحن ({current_optimal_cost:.2f}$) لا تزال مرتفعة وتتجاوز العتبة المقبولة ({cost_threshold}$) بمقدار {cost_exceedance:.2f}$.")
                mitigations.append("- إعادة تقييم هياكل التكلفة للمسارات الحالية (قد تخفض التكلفة بنسبة 5%).")
                mitigations.append("- استكشاف خيارات الشحن بالجملة أو عقود طويلة الأجل (قد تخفض التكلفة بنسبة 10-15%).")

    elif scenario_type == "marketing":
        budget_allocation = optimal_solution
        total_budget = current_params['total_budget']
        channel_roi_per_k = current_params['channel_roi_per_k']

        if budget_allocation is not None and total_budget > 0:
            # المخاطرة 1: التركيز المفرط على قناة واحدة
            channel_alloc_percents = [(alloc / total_budget) * 100 for alloc in budget_allocation]
            numerical_insights['social_media_alloc_percent'] = channel_alloc_percents[0]
            numerical_insights['paid_search_alloc_percent'] = channel_alloc_percents[1]
            numerical_insights['email_alloc_percent'] = channel_alloc_percents[2]

            if any(p > 70 for p in channel_alloc_percents): # إذا كان أكثر من 70% في قناة واحدة
                max_alloc_idx = np.argmax(channel_alloc_percents)
                channel_names = ['الشبكات الاجتماعية', 'البحث المدفوع', 'البريد الإلكتروني']
                risks.append(f"1. تركيز مفرط على قناة {channel_names[max_alloc_idx]} ({channel_alloc_percents[max_alloc_idx]:.2f}%)، مما يزيد من المخاطر في حالة تراجع أدائها أو تغير الخوارزميات.")
                mitigations.append("- تنويع الاستثمار عبر قنوات متعددة لتقليل المخاطر (قد يوزع 10-15% على قنوات أخرى).")
                mitigations.append("- مراقبة أداء القناة عالية التركيز عن كثب.")

            # المخاطرة 2: عدم تحقيق عائد استثمار مستهدف
            target_roi_sales_M = 1.25 # مليون دولار، هدف زيادة المبيعات بنسبة 25% من 500 ألف دولار ميزانية
                                      # نفترض أن المبيعات الأساسية كانت 5 مليون دولار، فزيادة 25% تعني 1.25 مليون دولار إضافية
            current_sales_M = sum(budget_allocation[i] * channel_roi_per_k[i] for i in range(len(budget_allocation))) / 1000 # تحويل إلى ملايين
            numerical_insights['current_optimal_sales_M'] = current_sales_M

            if current_sales_M < target_roi_sales_M:
                sales_deficit_M = target_roi_sales_M - current_sales_M
                numerical_insights['sales_deficit_M'] = sales_deficit_M
                risks.append(f"2. العائد المتوقع ({current_sales_M:.2f} مليون دولار) أقل من الهدف ({target_roi_sales_M:.2f} مليون دولار) بعجز قدره {sales_deficit_M:.2f} مليون دولار.")
                mitigations.append("- إعادة تقييم فعالية الحملات الحالية.")
                mitigations.append("- زيادة الميزانية الإجمالية (قد تحتاج لـ {sales_deficit_M * 1000 / np.mean(list(channel_roi_per_k)) if channel_roi_per_k else 1:.2f} ألف دولار إضافية).") # تم التعديل هنا
                mitigations.append("- استكشاف قنوات تسويقية جديدة ذات عائد أعلى.")

    if not risks:
        print("لا توجد مخاطر كبيرة تم تحديدها بناءً على التحليل الحالي.")
    else:
        print("\n--- المخاطر المحددة بواسطة الذكاء الاصطناعي (رقمية) ---")
        for key, value in numerical_insights.items():
            if 'percent' in key:
                print(f"- {key.replace('_', ' ')}: {value:.2f}%")
            elif '_M' in key:
                print(f"- {key.replace('_', ' ')}: ${value:.2f} مليون")
            else:
                print(f"- {key.replace('_', ' ')}: {value:.2f} وحدة/دولار")

        print("\n--- المخاطر المحددة بواسطة الذكاء الاصطناعي (وصفية) ---")
        for r in risks:
            print(r)

        print("\n--- استراتيجيات التخفيف المقترحة (مع تأثيرات تقديرية) ---")
        for m in mitigations:
            print(m)

    return numerical_insights

# -----------------------------------------------------------------------------
# مثال 1: تحسين سلسلة التوريد
# -----------------------------------------------------------------------------
print("=====================================================")
print("=== مثال 1: تحسين سلسلة التوريد (شحن المنتجات) ===")
print("=====================================================")

# شركة صناعية تحتاج إلى شحن عدد معين من المنتجات (سيتم تحديده بواسطة AI). لديها خياران للشحن:
# - المسار 1: التكلفة 5 دولارات للوحدة، يستغرق 3 أيام للوحدة. السعة القصوى 100 وحدة.
# - المسار 2: التكلفة 4 دولارات للوحدة، يستغرق 5 أيام للوحدة. السعة القصوى 120 وحدة.

# الهدف: تقليل إجمالي تكاليف الشحن مع ضمان متوسط وقت تسليم لا يتجاوز 4 أيام.

# تعريف المشكلة:
# المتغيرات: x1 = عدد الوحدات المشحونة عبر المسار 1، x2 = عدد الوحدات المشحونة عبر المسار 2

# دالة الهدف (لتقليل 5x1 + 4x2):
objective_coeffs_supply_chain = [5, 4] # معاملات التكلفة (للتصغير، لذا تبقى موجبة)
path_costs_supply_chain = [5, 4] # تكاليف المسارات لاستخدامها في تقييم المخاطر

# استخدام الذكاء الاصطناعي للتنبؤ بالطلب الإجمالي وتقديم رؤى
sample_historical_demand = [130, 140, 150, 155, 165] # بيانات طلب تاريخية فعلية
total_units_to_ship_ai = simulate_ai_insight(scenario_type="demand_prediction", historical_data=sample_historical_demand)['predicted_value'] # تم التعديل هنا

# قيود الموارد (A_ub * x <= b_ub):
# 1. تلبية الطلب الإجمالي (x1 + x2 >= total_units_to_ship_ai) -> -x1 - x2 <= -total_units_to_ship_ai
# 2. متوسط وقت التسليم (3x1 + 5x2) / (x1 + x2) <= 4
#    يمكن تحويلها إلى: 3x1 + 5x2 <= 4x1 + 4x2
#    أو: (3-4)x1 + (5-4)x2 <= 0
#    أو: -x1 + x2 <= 0
# 3. سعة المسار 1 (x1 <= 100)
# 4. سعة المسار 2 (x2 <= 120)
resource_matrix_supply_chain = [
    [-1, -1],  # قيد تلبية الطلب (>=)
    [-1, 1],   # قيد متوسط وقت التسليم (<=)
    [1, 0],    # قيد سعة المسار 1 (<=)
    [0, 1]     # قيد سعة المسار 2 (<=)
]
# تحديث قيد الطلب بناءً على توقع الذكاء الاصطناعي
available_resources_limits_supply_chain = [-total_units_to_ship_ai, 0, 100, 120] # حدود القيود
path_capacities_supply_chain = [100, 120] # سعات المسارات لاستخدامها في تقييم المخاطر

# حدود المتغيرات (x1 >= 0, x2 >= 0):
bounds_supply_chain = [(0, None), (0, None)] # (الحد الأدنى، الحد الأقصى)

# تشغيل المحسّن
results_supply_chain = optimize_resources_mvp(
    objective_coeffs_supply_chain,
    resource_matrix_supply_chain,
    available_resources_limits_supply_chain,
    bounds_supply_chain
)

# عرض النتائج
print("\n--- نتائج تحسين سلسلة التوريد (MVP) ---")
print(f"الحالة: {results_supply_chain['status']}")

if results_supply_chain['status'] == 'optimal':
    print(f"القيم المثلى للمتغيرات (عدد الوحدات المشحونة):")
    print(f"  الوحدات عبر المسار 1 (x1): {results_supply_chain['optimal_values'][0]:.2f} وحدة")
    print(f"  الوحدات عبر المسار 2 (x2): {results_supply_chain['optimal_values'][1]:.2f} وحدة")
    print(f"إجمالي التكلفة الدنيا للشحن: ${results_supply_chain['optimal_objective_value']:.2f}")

    # حساب التكلفة قبل التحسين (سيناريو افتراضي غير محسّن)
    units_via_path2_unoptimized = min(120, total_units_to_ship_ai)
    units_via_path1_unoptimized = total_units_to_ship_ai - units_via_path2_unoptimized

    if units_via_path1_unoptimized > 100 or units_via_path2_unoptimized > 120:
        cost_before_optimization_supply_chain = (100 * 5) + (120 * 4) # تكلفة ملء المسارين بالكامل كقيمة مرجعية
        print("\nملاحظة: السيناريو غير المحسّن الافتراضي تجاوز سعة المسار 1 أو 2. تم استخدام قيمة تقديرية للتكلفة قبل التحسين.")
    else:
        cost_before_optimization_supply_chain = (units_via_path1_unoptimized * 5) + (units_via_path2_unoptimized * 4)

    print(f"إجمالي التكلفة قبل التحسين (تقديري): ${cost_before_optimization_supply_chain:.2f}")

    # عرض النتائج كرسم بياني نصي
    labels_supply_chain = ['التكلفة قبل التحسين', 'التكلفة بعد التحسين']
    costs_supply_chain = [cost_before_optimization_supply_chain, results_supply_chain['optimal_objective_value']]
    print_ascii_bar_chart(labels_supply_chain, costs_supply_chain, "مقارنة تكاليف الشحن")

    # تشغيل إدارة المخاطر بعد الحصول على الحل الأمثل
    risk_insights_supply_chain = assess_and_mitigate_risks_ai(
        scenario_type="supply_chain",
        optimal_solution=results_supply_chain['optimal_values'],
        current_params={
            'predicted_demand': total_units_to_ship_ai,
            'path_costs': path_costs_supply_chain,
            'path_capacities': path_capacities_supply_chain
        }
    )

else:
    print(f"لم يتم العثور على حل أمثل. الرسالة: {results_supply_chain['message']}")

# -----------------------------------------------------------------------------
# مثال 2: التسويق: زيادة العائد على الاستثمار
# -----------------------------------------------------------------------------
print("\n\n=====================================================")
print("=== مثال 2: التسويق: زيادة العائد على الاستثمار ===")
print("=====================================================")

# شركة تجزئة ترغب في تخصيص ميزانية تسويق بقيمة 500 ألف دولار على قنوات متعددة:
# - الشبكات الاجتماعية: مبيعات متوقعة 1.5 دولار لكل دولار استثمار.
# - البحث المدفوع: مبيعات متوقعة 2.0 دولار لكل دولار استثمار.
# - البريد الإلكتروني: مبيعات متوقعة 1.0 دولار لكل دولار استثمار.

# الهدف: تعظيم إجمالي المبيعات المحققة.
# القيود:
# - الميزانية الإجمالية: 500 ألف دولار.
# - الحد الأدنى للإنفاق: الشبكات الاجتماعية 50 ألف، البحث المدفوع 100 ألف، البريد الإلكتروني 20 ألف.
# - الحد الأقصى للإنفاق: الشبكات الاجتماعية 250 ألف، البحث المدفوع 300 ألف، البريد الإلكتروني 150 ألف.

# تعريف المشكلة:
# المتغيرات: x1 = ميزانية الشبكات الاجتماعية (بالألف دولار)
#           x2 = ميزانية البحث المدفوع (بالألف دولار)
#           x3 = ميزانية البريد الإلكتروني (بالألف دولار)

# دالة الهدف (لتعظيم 1.5*x1 + 2.0*x2 + 1.0*x3):
# نستخدم معاملات سالبة لأن linprog تقوم بالتصغير
objective_coeffs_marketing = [-1.5, -2.0, -1.0]
channel_roi_per_k_marketing = [1.5, 2.0, 1.0] # تكاليف المسارات لاستخدامها في تقييم المخاطر (تم تحويلها إلى قائمة)

# قيود الموارد (A_ub * x <= b_ub و A_eq * x == b_eq):
# 1. الميزانية الإجمالية (x1 + x2 + x3 <= 500)
# 2. الحد الأدنى للإنفاق (x1 >= 50 -> -x1 <= -50)
# 3. الحد الأدنى للإنفاق (x2 >= 100 -> -x2 <= -100)
# 4. الحد الأدنى للإنفاق (x3 >= 20 -> -x3 <= -20)
# 5. الحد الأقصى للإنفاق (x1 <= 250)
# 6. الحد الأقصى للإنفاق (x2 <= 300)
# 7. الحد الأقصى للإنفاق (x3 <= 150)
resource_matrix_marketing = [
    [1, 1, 1],   # قيد الميزانية الإجمالية (<=)
    [-1, 0, 0],  # قيد الحد الأدنى للشبكات الاجتماعية (>=)
    [0, -1, 0],  # قيد الحد الأدنى للبحث المدفوع (>=)
    [0, 0, -1],  # قيد الحد الأدنى للبريد الإلكتروني (>=)
    [1, 0, 0],   # قيد الحد الأقصى للشبكات الاجتماعية (<=)
    [0, 1, 0],   # قيد الحد الأقصى للبحث المدفوع (<=)
    [0, 0, 1]    # قيد الحد الأقصى للبريد الإلكتروني (<=)
]
available_resources_limits_marketing = [500, -50, -100, -20, 250, 300, 150] # حدود القيود

# أنواع القيود (كلها <= في هذا المثال)
constraint_types_marketing = ['ub', 'ub', 'ub', 'ub', 'ub', 'ub', 'ub']

# حدود المتغيرات (x1, x2, x3 >= 0):
bounds_marketing = [(0, None), (0, None), (0, None)]

# تشغيل المحسّن
results_marketing = optimize_resources_mvp(
    objective_coeffs_marketing,
    resource_matrix_marketing,
    available_resources_limits_marketing,
    bounds_marketing,
    constraint_types_marketing
)

# عرض النتائج
print("\n--- نتائج تحسين ميزانية التسويق (MVP) ---")
print(f"الحالة: {results_marketing['status']}")

if results_marketing['status'] == 'optimal':
    print(f"القيم المثلى للمتغيرات (تخصيص الميزانية بالألف دولار):")
    print(f"  الشبكات الاجتماعية (x1): ${results_marketing['optimal_values'][0]:.2f} ألف")
    print(f"  البحث المدفوع (x2): ${results_marketing['optimal_values'][1]:.2f} ألف")
    print(f"  البريد الإلكتروني (x3): ${results_marketing['optimal_values'][2]:.2f} ألف")

    optimal_sales_marketing = results_marketing['optimal_objective_value'] # هذه هي المبيعات المحققة
    print(f"إجمالي المبيعات الأقصى المحققة: ${optimal_sales_marketing:.2f} ألف دولار")

    # حساب المبيعات قبل التحسين (سيناريو افتراضي - توزيع متساوٍ)
    total_budget_marketing = 500 # ألف دولار
    equal_allocation = total_budget_marketing / 3
    sales_before_optimization_marketing = (
        equal_allocation * channel_roi_per_k_marketing[0] + # تم التعديل هنا
        equal_allocation * channel_roi_per_k_marketing[1] + # تم التعديل هنا
        equal_allocation * channel_roi_per_k_marketing[2]   # تم التعديل هنا
    )
    print(f"إجمالي المبيعات قبل التحسين (توزيع متساوٍ تقديري): ${sales_before_optimization_marketing:.2f} ألف دولار")

    # نسبة الزيادة المتوقعة في المبيعات
    sales_increase_percent = ((optimal_sales_marketing - sales_before_optimization_marketing) / sales_before_optimization_marketing) * 100
    print(f"نسبة الزيادة المتوقعة في المبيعات بعد التحسين: {sales_increase_percent:.2f}%")

    # عرض النتائج كرسم بياني نصي
    labels_marketing = ['المبيعات قبل التحسين', 'المبيعات بعد التحسين']
    sales_values_marketing = [sales_before_optimization_marketing, optimal_sales_marketing]
    print_ascii_bar_chart(labels_marketing, sales_values_marketing, "مقارنة المبيعات التسويقية")

    # تشغيل إدارة المخاطر بعد الحصول على الحل الأمثل
    risk_insights_marketing = assess_and_mitigate_risks_ai(
        scenario_type="marketing",
        optimal_solution=results_marketing['optimal_values'],
        current_params={
            'total_budget': total_budget_marketing,
            'channel_roi_per_k': channel_roi_per_k_marketing # تم التعديل هنا
        }
    )

else:
    print(f"لم يتم العثور على حل أمثل. الرسالة: {results_marketing['message']}")


print("\n--- شرح إضافي عام ---")
print("يوضح هذا النموذج كيف يمكن لنظامنا أن يحدد أفضل طريقة لتوزيع الموارد (سواء كانت وحدات شحن أو ميزانيات تسويق) لتقليل التكاليف أو تعظيم الأرباح، مع ضمان تلبية الطلب وعدم تجاوز القيود.")
print("الرسوم البيانية النصية توضح بوضوح التوفير أو الزيادة في الأداء الذي يمكن تحقيقه من خلال تطبيق نموذج التحسين الخاص بنا.")
print("يمكن توسيع هذا المفهوم ليشمل عوامل أكثر تعقيدًا وتطبيقات أوسع في مختلف الصناعات.")
print("\n--- دمج الذكاء الاصطناعي ---")
print("في هذا التحديث، قمنا بدمج دالة 'simulate_ai_insight' التي تحاكي نموذج ذكاء اصطناعي يتنبأ بالطلب المستقبلي أو يقدم رؤى حول أداء القنوات التسويقية بناءً على البيانات التاريخية. هذه الرؤى تغذي بعد ذلك نموذج التحسين، مما يوضح كيف يمكن للذكاء الاصطناعي أن يوفر مدخلات ديناميكية للتحسين، مما يجعله أكثر استجابة للظروف المتغيرة. كما تقوم الدالة بتقديم مقترحات ورؤى بناءً على تحليلها لمساعدة في اتخاذ القرار الاستراتيجي.")
print("\n--- إدارة المخاطر المدعومة بالذكاء الاصطناعي ---")
print("تم إضافة دالة 'assess_and_mitigate_risks_ai' التي تقوم بمحاكاة تحليل المخاطر بناءً على الحل الأمثل والمدخلات الأخرى. تقوم هذه الدالة بتحديد المخاطر المحتملة (مثل الاعتماد المفرط على مسار واحد، أو عدم كفاية السعة، أو ارتفاع التكاليف، أو عدم تحقيق عائد استثمار مستهدف) وتقديم استراتيجيات تخفيف مقترحة، مع تضمين رؤى رقمية لتقييم المخاطر بشكل كمي، مما يوفر رؤى قيمة لاتخاذ قرارات أكثر قوة واستدامة.")


=== مثال 1: تحسين سلسلة التوريد (شحن المنتجات) ===

--- تشغيل نموذج الذكاء الاصطناعي لرؤى demand_prediction ---
البيانات التاريخية: [130, 140, 150, 155, 165]
المتوسط التاريخي: 148.00 وحدة
الاتجاه (لكل فترة): 7.00 وحدة
القيمة المتوقعة بواسطة الذكاء الاصطناعي: 162 وحدة

--- مقترحات الذكاء الاصطناعي للمساعدة في اتخاذ القرار ---
1. تحليل البيانات التاريخية يظهر اتجاهًا تصاعديًا طفيفًا.
2. بناءً على التنبؤ بـ 162 وحدة، يُنصح بمراجعة سعة المسارات.
3. قد تحتاج إلى زيادة الاعتماد على المسار 2، أو التفكير في توسيع سعة المسار 1.

--- نتائج تحسين سلسلة التوريد (MVP) ---
الحالة: optimal
القيم المثلى للمتغيرات (عدد الوحدات المشحونة):
  الوحدات عبر المسار 1 (x1): 81.00 وحدة
  الوحدات عبر المسار 2 (x2): 81.00 وحدة
إجمالي التكلفة الدنيا للشحن: $729.00
إجمالي التكلفة قبل التحسين (تقديري): $690.00

--- مقارنة تكاليف الشحن ---
التكلفة قبل التحسين | ███████████████████████████████████████████████ 690.00
التكلفة بعد التحسين | ██████████████████████████████████████████████████ 729.00
-----------------------

In [12]:
import numpy as np
from scipy.optimize import linprog

def optimize_resources_mvp(
    objective_coefficients,
    resource_constraints_matrix,
    available_resources,
    variable_bounds=None,
    constraint_types=None # 'eq' for equality, 'ub' for upper bound
):
    """
    يقوم بتحسين الأهداف بموارد محدودة تحت قيود باستخدام البرمجة الخطية.

    هذا هو نموذج MVP بسيط يوضح كيفية عمل إطار التحسين.
    الهدف هو تعظيم دالة الهدف الخطية مع الالتزام بالقيود الخطية.

    المعاملات:
    objective_coefficients (list or array): معاملات دالة الهدف.
                                           ملاحظة: linprog تقوم بالتصغير، لذا إذا كنت تريد التعظيم،
                                           اجعل هذه المعاملات سالبة.
                                           (مثال: لتعظيم 30x + 40y، استخدم [-30, -40]).
    resource_constraints_matrix (list of lists or 2D array): مصفوفة معاملات قيود الموارد.
                                                              كل صف يمثل قيدًا، وكل عمود يمثل متغيرًا.
    available_resources (list or array): القيم القصوى للموارد المتاحة لكل قيد.
                                        (الجانب الأيمن من القيود).
    variable_bounds (list of tuples, optional): حدود كل متغير.
                                                افتراضيًا، يتم تعيينها على (0, None) مما يعني أن المتغيرات غير سالبة.
                                                (مثال: [(0, None), (0, None)] لمتغيرين غير سالبين).
    constraint_types (list of str, optional): نوع كل قيد في resource_constraints_matrix.
                                              'ub' لـ <= (أقل من أو يساوي)، 'eq' لـ == (يساوي).
                                              الافتراضي هو 'ub' للجميع.

    العودة:
    dict: قاموس يحتوي على النتائج:
          - 'status': حالة الحل (مثال: 'optimal', 'infeasible').
          - 'optimal_values': القيم المثلى للمتغيرات.
          - 'optimal_objective_value': القيمة المثلى لدالة الهدف (بعد تحويلها للتعظيم إذا كانت المعاملات سالبة).
          - 'message': رسالة من المحلّل.
    """

    c = np.array(objective_coefficients)
    A_ub = []
    b_ub = []
    A_eq = []
    b_eq = []

    if constraint_types is None:
        constraint_types = ['ub'] * len(resource_constraints_matrix)

    for i, row in enumerate(resource_constraints_matrix):
        if constraint_types[i] == 'ub':
            A_ub.append(row)
            b_ub.append(available_resources[i])
        elif constraint_types[i] == 'eq':
            A_eq.append(row)
            b_eq.append(available_resources[i])
        else:
            raise ValueError("Unsupported constraint type. Use 'ub' or 'eq'.")

    A_ub = np.array(A_ub) if A_ub else None
    b_ub = np.array(b_ub) if b_ub else None
    A_eq = np.array(A_eq) if A_eq else None
    b_eq = np.array(b_eq) if b_eq else None

    if variable_bounds is None:
        variable_bounds = [(0, None)] * len(c)

    result = linprog(c, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq, bounds=variable_bounds, method='highs')

    if result.success:
        optimal_values = result.x
        optimal_objective_value = -result.fun if np.all(c < 0) else result.fun
        status = 'optimal'
    else:
        optimal_values = None
        optimal_objective_value = None
        status = result.status

    return {
        'status': status,
        'optimal_values': optimal_values,
        'optimal_objective_value': optimal_objective_value,
        'message': result.message
    }

def print_ascii_bar_chart(labels, values, title=""):
    """
    يطبع رسم بياني شريطي نصي (ASCII art).
    """
    print(f"\n--- {title} ---")
    if not values:
        print("لا توجد بيانات لعرضها.")
        return

    max_label_length = max(len(label) for label in labels)
    max_value = max(values)

    if max_value == 0:
        scale = 0
    else:
        scale = 50 / max_value

    for i, label in enumerate(labels):
        value = values[i]
        bar_length = int(value * scale)
        bar = '█' * bar_length
        print(f"{label.ljust(max_label_length)} | {bar} {value:.2f}")
    print("-" * (max_label_length + 55))

def simulate_ai_insight(scenario_type, historical_data=None):
    """
    دالة محاكاة لنموذج ذكاء اصطناعي يقدم رؤى وتنبؤات بناءً على نوع السيناريو.
    """
    print(f"\n--- تشغيل نموذج الذكاء الاصطناعي لرؤى {scenario_type} ---")
    insights = {}
    suggestions = []

    if scenario_type == "demand_prediction":
        if historical_data is None or not historical_data:
            historical_data = [140, 150, 145, 155, 160] # بيانات طلب تاريخية افتراضية

        avg_data = np.mean(historical_data)
        data_trend = (historical_data[-1] - historical_data[0]) / len(historical_data) if len(historical_data) > 1 else 0
        predicted_value = int(avg_data + data_trend * 2)

        insights['historical_data'] = historical_data
        insights['average_historical_value'] = avg_data
        insights['trend_per_period'] = data_trend
        insights['predicted_value'] = predicted_value

        print(f"البيانات التاريخية: {historical_data}")
        print(f"المتوسط التاريخي: {avg_data:.2f} وحدة")
        print(f"الاتجاه (لكل فترة): {data_trend:.2f} وحدة")
        print(f"القيمة المتوقعة بواسطة الذكاء الاصطناعي: {predicted_value} وحدة")

        suggestions.append("1. تحليل البيانات التاريخية للطلب يظهر اتجاهًا تصاعديًا طفيفًا.")
        suggestions.append(f"2. بناءً على التنبؤ بـ {predicted_value} وحدة، يُنصح بمراجعة سعة المسارات.")
        if predicted_value > 100:
            suggestions.append("3. قد تحتاج إلى زيادة الاعتماد على المسار 2، أو التفكير في توسيع سعة المسار 1.")
        if predicted_value > 200:
            suggestions.append("4. يُقترح استكشاف خيارات شحن جديدة أو التفاوض على عقود أفضل مع الموردين الحاليين لتلبية الطلب المتزايد.")

    elif scenario_type == "marketing_roi_potential":
        # محاكاة بيانات الأداء التاريخي للقنوات التسويقية (ROI لكل 1000 دولار استثمار)
        channel_performance = {
            'social_media_sales_per_k': 1500, # $1500 مبيعات لكل $1000 استثمار
            'paid_search_sales_per_k': 2000,
            'email_sales_per_k': 1000
        }

        insights['channel_performance_metrics'] = channel_performance

        total_budget_k = 500 # 500 ألف دولار
        estimated_total_sales_potential = (
            channel_performance['social_media_sales_per_k'] * (total_budget_k / 3) +
            channel_performance['paid_search_sales_per_k'] * (total_budget_k / 3) +
            channel_performance['email_sales_per_k'] * (total_budget_k / 3)
        ) / 1000 # تحويل إلى ملايين الدولارات

        insights['estimated_total_sales_potential_M'] = estimated_total_sales_potential

        print(f"الأداء المتوقع للقنوات (مبيعات لكل 1000$ استثمار): {channel_performance}")
        print(f"إجمالي المبيعات المحتملة المقدرة (قبل التحسين الأمثل): ${estimated_total_sales_potential:.2f} مليون")

        suggestions.append("1. البحث المدفوع يظهر أعلى عائد محتمل لكل دولار مستثمر.")
        suggestions.append("2. يمكن تحسين إجمالي المبيعات بتوزيع الميزانية بناءً على أداء القناة.")
        suggestions.append("3. يُنصح بمراقبة أداء القنوات باستمرار لتعديل التخصيص.")

    print("\n--- مقترحات الذكاء الاصطناعي للمساعدة في اتخاذ القرار ---")
    for s in suggestions:
        print(s)

    return insights

def assess_and_mitigate_risks_ai(scenario_type, optimal_solution, current_params, historical_data=None):
    """
    دالة محاكاة لنموذج ذكاء اصطناعي يقوم بتقييم المخاطر وتقديم استراتيجيات التخفيف.
    """
    print(f"\n--- تشغيل نموذج الذكاء الاصطناعي لإدارة المخاطر لـ {scenario_type} ---")
    risks = []
    mitigations = []
    numerical_insights = {}

    if scenario_type == "supply_chain":
        predicted_demand = current_params['predicted_demand']
        path_costs = current_params['path_costs']
        path_capacities = current_params['path_capacities']

        # المخاطرة 1: الاعتماد المفرط على مسار واحد
        if optimal_solution is not None:
            total_shipped = sum(optimal_solution)
            if total_shipped > 0:
                path1_reliance_percent = (optimal_solution[0] / total_shipped) * 100
                path2_reliance_percent = (optimal_solution[1] / total_shipped) * 100

                numerical_insights['path1_reliance_percent'] = path1_reliance_percent
                numerical_insights['path2_reliance_percent'] = path2_reliance_percent

                if path1_reliance_percent > 80:
                    risks.append(f"1. اعتماد كبير على المسار 1 ({path1_reliance_percent:.2f}%), مما يزيد من المخاطر في حالة تعطله.")
                    mitigations.append("- تنويع الشحنات قدر الإمكان بين المسارات لتقليل المخاطر (قد يقلل الاعتماد بنسبة 5-10%).")
                    mitigations.append("- وضع خطة طوارئ للمسار 1 (بدائل، عقود احتياطية).")
                elif path2_reliance_percent > 80:
                    risks.append(f"1. اعتماد كبير على المسار 2 ({path2_reliance_percent:.2f}%), مما يزيد من المخاطر في حالة تعطله.")
                    mitigations.append("- تنويع الشحنات قدر الإمكان بين المسارات لتقليل المخاطر (قد يقلل الاعتماد بنسبة 5-10%).")
                    mitigations.append("- وضع خطة طوارئ للمسار 2 (بدائل، عقود احتياطية).")

        # المخاطرة 2: عدم كفاية السعة لتلبية الطلب المتوقع
        total_capacity = sum(path_capacities)
        if predicted_demand > total_capacity:
            capacity_deficit = predicted_demand - total_capacity
            numerical_insights['capacity_deficit'] = capacity_deficit
            risks.append(f"2. السعة الإجمالية للمسارات ({total_capacity} وحدة) أقل من الطلب المتوقع ({predicted_demand} وحدة) بعجز قدره {capacity_deficit} وحدة.")
            mitigations.append(f"- التفاوض على زيادة السعة مع الموردين الحاليين (يمكن أن يزيد السعة بنسبة 10-20%).")
            mitigations.append("- البحث عن مسارات شحن إضافية أو شركاء جدد.")
            mitigations.append("- النظر في تعديل تواريخ التسليم أو الطلبات مع العملاء.")

        # المخاطرة 3: ارتفاع تكاليف الشحن (إذا كانت التكلفة المثلى لا تزال مرتفعة)
        cost_threshold = 650
        if optimal_solution is not None:
            current_optimal_cost = optimal_solution[0] * path_costs[0] + optimal_solution[1] * path_costs[1]
            if current_optimal_cost > cost_threshold:
                cost_exceedance = current_optimal_cost - cost_threshold
                numerical_insights['cost_exceedance'] = cost_exceedance
                risks.append(f"3. التكاليف المثلى للشحن ({current_optimal_cost:.2f}$) لا تزال مرتفعة وتتجاوز العتبة المقبولة ({cost_threshold}$) بمقدار {cost_exceedance:.2f}$.")
                mitigations.append("- إعادة تقييم هياكل التكلفة للمسارات الحالية (قد تخفض التكلفة بنسبة 5%).")
                mitigations.append("- استكشاف خيارات الشحن بالجملة أو عقود طويلة الأجل (قد تخفض التكلفة بنسبة 10-15%).")

    elif scenario_type == "marketing":
        budget_allocation = optimal_solution
        total_budget = current_params['total_budget']
        channel_roi_per_k = current_params['channel_roi_per_k']

        if budget_allocation is not None and total_budget > 0:
            # المخاطرة 1: التركيز المفرط على قناة واحدة
            channel_alloc_percents = [(alloc / total_budget) * 100 for alloc in budget_allocation]
            numerical_insights['social_media_alloc_percent'] = channel_alloc_percents[0]
            numerical_insights['paid_search_alloc_percent'] = channel_alloc_percents[1]
            numerical_insights['email_alloc_percent'] = channel_alloc_percents[2]

            if any(p > 70 for p in channel_alloc_percents):
                max_alloc_idx = np.argmax(channel_alloc_percents)
                channel_names = ['الشبكات الاجتماعية', 'البحث المدفوع', 'البريد الإلكتروني']
                risks.append(f"1. تركيز مفرط على قناة {channel_names[max_alloc_idx]} ({channel_alloc_percents[max_alloc_idx]:.2f}%)، مما يزيد من المخاطر في حالة تراجع أدائها أو تغير الخوارزميات.")
                mitigations.append("- تنويع الاستثمار عبر قنوات متعددة لتقليل المخاطر (قد يوزع 10-15% على قنوات أخرى).")
                mitigations.append("- مراقبة أداء القناة عالية التركيز عن كثب.")

            # المخاطرة 2: عدم تحقيق عائد استثمار مستهدف
            target_roi_sales_M = 1.25
            current_sales_M = sum(budget_allocation[i] * channel_roi_per_k[i] for i in range(len(budget_allocation))) / 1000
            numerical_insights['current_optimal_sales_M'] = current_sales_M

            if current_sales_M < target_roi_sales_M:
                sales_deficit_M = target_roi_sales_M - current_sales_M
                numerical_insights['sales_deficit_M'] = sales_deficit_M
                risks.append(f"2. العائد المتوقع ({current_sales_M:.2f} مليون دولار) أقل من الهدف ({target_roi_sales_M:.2f} مليون دولار) بعجز قدره {sales_deficit_M:.2f} مليون دولار.")
                mitigations.append("- إعادة تقييم فعالية الحملات الحالية.")
                # تأكد من أن channel_roi_per_k ليست فارغة لتجنب القسمة على صفر
                avg_roi_per_k = np.mean(channel_roi_per_k) if channel_roi_per_k else 1
                mitigations.append(f"- زيادة الميزانية الإجمالية (قد تحتاج لـ {sales_deficit_M * 1000 / avg_roi_per_k:.2f} ألف دولار إضافية).")
                mitigations.append("- استكشاف قنوات تسويقية جديدة ذات عائد أعلى.")

    if not risks:
        print("لا توجد مخاطر كبيرة تم تحديدها بناءً على التحليل الحالي.")
    else:
        print("\n--- المخاطر المحددة بواسطة الذكاء الاصطناعي (رقمية) ---")
        for key, value in numerical_insights.items():
            if 'percent' in key:
                print(f"- {key.replace('_', ' ')}: {value:.2f}%")
            elif '_M' in key:
                print(f"- {key.replace('_', ' ')}: ${value:.2f} مليون")
            else:
                print(f"- {key.replace('_', ' ')}: {value:.2f} وحدة/دولار")

        print("\n--- المخاطر المحددة بواسطة الذكاء الاصطناعي (وصفية) ---")
        for r in risks:
            print(r)

        print("\n--- استراتيجيات التخفيف المقترحة (مع تأثيرات تقديرية) ---")
        for m in mitigations:
            print(m)

    return numerical_insights

# -----------------------------------------------------------------------------
# مثال 1: تحسين سلسلة التوريد
# -----------------------------------------------------------------------------
print("=====================================================")
print("=== مثال 1: تحسين سلسلة التوريد (شحن المنتجات) ===")
print("=====================================================")

# شركة صناعية تحتاج إلى شحن عدد معين من المنتجات (سيتم تحديده بواسطة AI). لديها خياران للشحن:
# - المسار 1: التكلفة 5 دولارات للوحدة، يستغرق 3 أيام للوحدة. السعة القصوى 100 وحدة.
# - المسار 2: التكلفة 4 دولارات للوحدة، يستغرق 5 أيام للوحدة. السعة القصوى 120 وحدة.

# الهدف: تقليل إجمالي تكاليف الشحن مع ضمان متوسط وقت تسليم لا يتجاوز 4 أيام.

# تعريف المشكلة:
# المتغيرات: x1 = عدد الوحدات المشحونة عبر المسار 1، x2 = عدد الوحدات المشحونة عبر المسار 2

# دالة الهدف (لتقليل 5x1 + 4x2):
objective_coeffs_supply_chain = [5, 4] # معاملات التكلفة (للتصغير، لذا تبقى موجبة)
path_costs_supply_chain = [5, 4] # تكاليف المسارات لاستخدامها في تقييم المخاطر

# استخدام الذكاء الاصطناعي للتنبؤ بالطلب الإجمالي وتقديم رؤى
demand_insights = simulate_ai_insight(scenario_type="demand_prediction", historical_data=[130, 140, 150, 155, 165])
total_units_to_ship_ai = demand_insights['predicted_value']

# قيود الموارد (A_ub * x <= b_ub):
# 1. تلبية الطلب الإجمالي (x1 + x2 >= total_units_to_ship_ai) -> -x1 - x2 <= -total_units_to_ship_ai
# 2. متوسط وقت التسليم (3x1 + 5x2) / (x1 + x2) <= 4
#    يمكن تحويلها إلى: 3x1 + 5x2 <= 4x1 + 4x2
#    أو: (3-4)x1 + (5-4)x2 <= 0
#    أو: -x1 + x2 <= 0
# 3. سعة المسار 1 (x1 <= 100)
# 4. سعة المسار 2 (x2 <= 120)
resource_matrix_supply_chain = [
    [-1, -1],  # قيد تلبية الطلب (>=)
    [-1, 1],   # قيد متوسط وقت التسليم (<=)
    [1, 0],    # قيد سعة المسار 1 (<=)
    [0, 1]     # قيد سعة المسار 2 (<=)
]
# تحديث قيد الطلب بناءً على توقع الذكاء الاصطناعي
available_resources_limits_supply_chain = [-total_units_to_ship_ai, 0, 100, 120] # حدود القيود
path_capacities_supply_chain = [100, 120] # سعات المسارات لاستخدامها في تقييم المخاطر

# حدود المتغيرات (x1 >= 0, x2 >= 0):
bounds_supply_chain = [(0, None), (0, None)] # (الحد الأدنى، الحد الأقصى)

# تشغيل المحسّن
results_supply_chain = optimize_resources_mvp(
    objective_coeffs_supply_chain,
    resource_matrix_supply_chain,
    available_resources_limits_supply_chain,
    bounds_supply_chain
)

# عرض النتائج
print("\n--- نتائج تحسين سلسلة التوريد (MVP) ---")
print(f"الحالة: {results_supply_chain['status']}")

if results_supply_chain['status'] == 'optimal':
    print(f"القيم المثلى للمتغيرات (عدد الوحدات المشحونة):")
    print(f"  الوحدات عبر المسار 1 (x1): {results_supply_chain['optimal_values'][0]:.2f} وحدة")
    print(f"  الوحدات عبر المسار 2 (x2): {results_supply_chain['optimal_values'][1]:.2f} وحدة")
    print(f"إجمالي التكلفة الدنيا للشحن: ${results_supply_chain['optimal_objective_value']:.2f}")

    # حساب التكلفة قبل التحسين (سيناريو افتراضي غير محسّن)
    units_via_path2_unoptimized = min(120, total_units_to_ship_ai)
    units_via_path1_unoptimized = total_units_to_ship_ai - units_via_path2_unoptimized

    if units_via_path1_unoptimized > 100 or units_via_path2_unoptimized > 120:
        cost_before_optimization_supply_chain = (100 * 5) + (120 * 4) # تكلفة ملء المسارين بالكامل كقيمة مرجعية
        print("\nملاحظة: السيناريو غير المحسّن الافتراضي تجاوز سعة المسار 1 أو 2. تم استخدام قيمة تقديرية للتكلفة قبل التحسين.")
    else:
        cost_before_optimization_supply_chain = (units_via_path1_unoptimized * 5) + (units_via_path2_unoptimized * 4)

    print(f"إجمالي التكلفة قبل التحسين (تقديري): ${cost_before_optimization_supply_chain:.2f}")

    # عرض النتائج كرسم بياني نصي
    labels_supply_chain = ['التكلفة قبل التحسين', 'التكلفة بعد التحسين']
    costs_supply_chain = [cost_before_optimization_supply_chain, results_supply_chain['optimal_objective_value']]
    print_ascii_bar_chart(labels_supply_chain, costs_supply_chain, "مقارنة تكاليف الشحن")

    # تشغيل إدارة المخاطر بعد الحصول على الحل الأمثل
    risk_insights_supply_chain = assess_and_mitigate_risks_ai(
        scenario_type="supply_chain",
        optimal_solution=results_supply_chain['optimal_values'],
        current_params={
            'predicted_demand': total_units_to_ship_ai,
            'path_costs': path_costs_supply_chain,
            'path_capacities': path_capacities_supply_chain
        }
    )

else:
    print(f"لم يتم العثور على حل أمثل. الرسالة: {results_supply_chain['message']}")

# -----------------------------------------------------------------------------
# مثال 2: التسويق: زيادة العائد على الاستثمار
# -----------------------------------------------------------------------------
print("\n\n=====================================================")
print("=== مثال 2: التسويق: زيادة العائد على الاستثمار ===")
print("=====================================================")

# شركة تجزئة ترغب في تخصيص ميزانية تسويق بقيمة 500 ألف دولار على قنوات متعددة:
# - الشبكات الاجتماعية: مبيعات متوقعة 1.5 دولار لكل دولار استثمار.
# - البحث المدفوع: مبيعات متوقعة 2.0 دولار لكل دولار استثمار.
# - البريد الإلكتروني: مبيعات متوقعة 1.0 دولار لكل دولار استثمار.

# الهدف: تعظيم إجمالي المبيعات المحققة.
# القيود:
# - الميزانية الإجمالية: 500 ألف دولار.
# - الحد الأدنى للإنفاق: الشبكات الاجتماعية 50 ألف، البحث المدفوع 100 ألف، البريد الإلكتروني 20 ألف.
# - الحد الأقصى للإنفاق: الشبكات الاجتماعية 250 ألف، البحث المدفوع 300 ألف، البريد الإلكتروني 150 ألف.

# تعريف المشكلة:
# المتغيرات: x1 = ميزانية الشبكات الاجتماعية (بالألف دولار)
#           x2 = ميزانية البحث المدفوع (بالألف دولار)
#           x3 = ميزانية البريد الإلكتروني (بالألف دولار)

# دالة الهدف (لتعظيم 1.5*x1 + 2.0*x2 + 1.0*x3):
# نستخدم معاملات سالبة لأن linprog تقوم بالتصغير
objective_coeffs_marketing = [-1.5, -2.0, -1.0]
channel_roi_per_k_marketing = [1.5, 2.0, 1.0] # لأغراض المخاطر

# قيود الموارد (A_ub * x <= b_ub و A_eq * x == b_eq):
# 1. الميزانية الإجمالية (x1 + x2 + x3 <= 500)
# 2. الحد الأدنى للإنفاق (x1 >= 50 -> -x1 <= -50)
# 3. الحد الأدنى للإنفاق (x2 >= 100 -> -x2 <= -100)
# 4. الحد الأدنى للإنفاق (x3 >= 20 -> -x3 <= -20)
# 5. الحد الأقصى للإنفاق (x1 <= 250)
# 6. الحد الأقصى للإنفاق (x2 <= 300)
# 7. الحد الأقصى للإنفاق (x3 <= 150)
resource_matrix_marketing = [
    [1, 1, 1],   # قيد الميزانية الإجمالية (<=)
    [-1, 0, 0],  # قيد الحد الأدنى للشبكات الاجتماعية (>=)
    [0, -1, 0],  # قيد الحد الأدنى للبحث المدفوع (>=)
    [0, 0, -1],  # قيد الحد الأدنى للبريد الإلكتروني (>=)
    [1, 0, 0],   # قيد الحد الأقصى للشبكات الاجتماعية (<=)
    [0, 1, 0],   # قيد الحد الأقصى للبحث المدفوع (<=)
    [0, 0, 1]    # قيد الحد الأقصى للبريد الإلكتروني (<=)
]
available_resources_limits_marketing = [500, -50, -100, -20, 250, 300, 150] # حدود القيود

# أنواع القيود (كلها <= في هذا المثال)
constraint_types_marketing = ['ub', 'ub', 'ub', 'ub', 'ub', 'ub', 'ub']

# حدود المتغيرات (x1, x2, x3 >= 0):
bounds_marketing = [(0, None), (0, None), (0, None)]

# تشغيل المحسّن
results_marketing = optimize_resources_mvp(
    objective_coeffs_marketing,
    resource_matrix_marketing,
    available_resources_limits_marketing,
    bounds_marketing,
    constraint_types_marketing
)

# عرض النتائج
print("\n--- نتائج تحسين ميزانية التسويق (MVP) ---")
print(f"الحالة: {results_marketing['status']}")

if results_marketing['status'] == 'optimal':
    print(f"القيم المثلى للمتغيرات (تخصيص الميزانية بالألف دولار):")
    print(f"  الشبكات الاجتماعية (x1): ${results_marketing['optimal_values'][0]:.2f} ألف")
    print(f"  البحث المدفوع (x2): ${results_marketing['optimal_values'][1]:.2f} ألف")
    print(f"  البريد الإلكتروني (x3): ${results_marketing['optimal_values'][2]:.2f} ألف")

    optimal_sales_marketing = results_marketing['optimal_objective_value'] # هذه هي المبيعات المحققة
    print(f"إجمالي المبيعات الأقصى المحققة: ${optimal_sales_marketing:.2f} ألف دولار")

    # حساب المبيعات قبل التحسين (سيناريو افتراضي - توزيع متساوٍ)
    total_budget_marketing = 500 # ألف دولار
    equal_allocation = total_budget_marketing / 3
    sales_before_optimization_marketing = (
        equal_allocation * channel_roi_per_k_marketing[0] +
        equal_allocation * channel_roi_per_k_marketing[1] +
        equal_allocation * channel_roi_per_k_marketing[2]
    )
    print(f"إجمالي المبيعات قبل التحسين (توزيع متساوٍ تقديري): ${sales_before_optimization_marketing:.2f} ألف دولار")

    # نسبة الزيادة المتوقعة في المبيعات
    sales_increase_percent = ((optimal_sales_marketing - sales_before_optimization_marketing) / sales_before_optimization_marketing) * 100
    print(f"نسبة الزيادة المتوقعة في المبيعات بعد التحسين: {sales_increase_percent:.2f}%")

    # عرض النتائج كرسم بياني نصي
    labels_marketing = ['المبيعات قبل التحسين', 'المبيعات بعد التحسين']
    sales_values_marketing = [sales_before_optimization_marketing, optimal_sales_marketing]
    print_ascii_bar_chart(labels_marketing, sales_values_marketing, "مقارنة المبيعات التسويقية")

    # تشغيل إدارة المخاطر بعد الحصول على الحل الأمثل
    risk_insights_marketing = assess_and_mitigate_risks_ai(
        scenario_type="marketing",
        optimal_solution=results_marketing['optimal_values'],
        current_params={
            'total_budget': total_budget_marketing,
            'channel_roi_per_k': channel_roi_per_k_marketing
        }
    )

else:
    print(f"لم يتم العثور على حل أمثل. الرسالة: {results_marketing['message']}")


print("\n--- شرح إضافي عام ---")
print("يوضح هذا النموذج كيف يمكن لنظامنا أن يحدد أفضل طريقة لتوزيع الموارد (سواء كانت وحدات شحن أو ميزانيات تسويق) لتقليل التكاليف أو تعظيم الأرباح، مع ضمان تلبية الطلب وعدم تجاوز القيود.")
print("الرسوم البيانية النصية توضح بوضوح التوفير أو الزيادة في الأداء الذي يمكن تحقيقه من خلال تطبيق نموذج التحسين الخاص بنا.")
print("يمكن توسيع هذا المفهوم ليشمل عوامل أكثر تعقيدًا وتطبيقات أوسع في مختلف الصناعات.")
print("\n--- دمج الذكاء الاصطناعي ---")
print("في هذا التحديث، قمنا بدمج دالة 'simulate_ai_insight' التي تحاكي نموذج ذكاء اصطناعي يتنبأ بالطلب المستقبلي أو يقدم رؤى حول أداء القنوات التسويقية بناءً على البيانات التاريخية. هذه الرؤى تغذي بعد ذلك نموذج التحسين، مما يوضح كيف يمكن للذكاء الاصطناعي أن يوفر مدخلات ديناميكية للتحسين، مما يجعله أكثر استجابة للظروف المتغيرة. كما تقوم الدالة بتقديم مقترحات ورؤى بناءً على تحليلها لمساعدة في اتخاذ القرار الاستراتيجي.")
print("\n--- إدارة المخاطر المدعومة بالذكاء الاصطناعي ---")
print("تم إضافة دالة 'assess_and_mitigate_risks_ai' التي تقوم بمحاكاة تحليل المخاطر بناءً على الحل الأمثل والمدخلات الأخرى. تقوم هذه الدالة بتحديد المخاطر المحتملة (مثل الاعتماد المفرط على مسار واحد، أو عدم كفاية السعة، أو ارتفاع التكاليف، أو عدم تحقيق عائد استثمار مستهدف) وتقديم استراتيجيات تخفيف مقترحة، مع تضمين رؤى رقمية لتقييم المخاطر بشكل كمي، مما يوفر رؤى قيمة لاتخاذ قرارات أكثر قوة واستدامة.")


=== مثال 1: تحسين سلسلة التوريد (شحن المنتجات) ===

--- تشغيل نموذج الذكاء الاصطناعي لرؤى demand_prediction ---
البيانات التاريخية: [130, 140, 150, 155, 165]
المتوسط التاريخي: 148.00 وحدة
الاتجاه (لكل فترة): 7.00 وحدة
القيمة المتوقعة بواسطة الذكاء الاصطناعي: 162 وحدة

--- مقترحات الذكاء الاصطناعي للمساعدة في اتخاذ القرار ---
1. تحليل البيانات التاريخية للطلب يظهر اتجاهًا تصاعديًا طفيفًا.
2. بناءً على التنبؤ بـ 162 وحدة، يُنصح بمراجعة سعة المسارات.
3. قد تحتاج إلى زيادة الاعتماد على المسار 2، أو التفكير في توسيع سعة المسار 1.

--- نتائج تحسين سلسلة التوريد (MVP) ---
الحالة: optimal
القيم المثلى للمتغيرات (عدد الوحدات المشحونة):
  الوحدات عبر المسار 1 (x1): 81.00 وحدة
  الوحدات عبر المسار 2 (x2): 81.00 وحدة
إجمالي التكلفة الدنيا للشحن: $729.00
إجمالي التكلفة قبل التحسين (تقديري): $690.00

--- مقارنة تكاليف الشحن ---
التكلفة قبل التحسين | ███████████████████████████████████████████████ 690.00
التكلفة بعد التحسين | ██████████████████████████████████████████████████ 729.00
-----------------